**Group Members: **

* Chandana Vangapalli (ID: 233323)
* Yizhou Yu (ID: 236819)
* Akpany Aubin Benjamin Ebby (ID: 232836 )
* Bangyi Wu (ID: 183398)


**Objective: **

The primary objective of our project is to apply sentiment analysis to the US airline industry twitter data to understand the trends in customer perceptions about some of the biggest airline companies in the USA. Further, we will also investigate what factors drive positive and negative sentiments and how this would impact the overall industry.  


**Data: **

We will use airline industry tweets for our analysis. Alternatively, we may use twitter datasets sourced from Kaggle.
